In [1]:
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import folium
import sys
import numpy as np


In [2]:
import os

# Print current working directory
print(f"Current directory: {os.getcwd()}")

# Change to parent directory
os.chdir('..')
print(f"Current directory: {os.getcwd()}")

Current directory: /Users/apple/Documents/GitHub/data_science_project/notebooks
Current directory: /Users/apple/Documents/GitHub/data_science_project


# Population

In [3]:
pop_dir=pd.read_excel("data/population.xlsx")

In [4]:
pop_dir.head(10)

,Geography,State,County,City,Population,Unnamed: 5,Unnamed: 6,Median Household Income,Average Household Income,Household Income Distribution,...,Unnamed: 58,Households,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Land Area (square miles),GEOID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Total,Families,Married Couple Families,NaN,Single Parent Families,NaN,Non-families (unrelated people sharing a home),NaN,NaN
1,NaN,NaN,NaN,NaN,Most Current Population Year,Population,Population Density (people per square mile),NaN,NaN,NaN,...,NaN,NaN,NaN,Total,With Kids Under 18 Years,Total,With Kids Under 18 Years,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Households,...,Median Value of Owner-Occupied Homes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,name,state,county,city,most_current_pop_year,population,pop_dens_sq_mi,mhhi,avghhi,hhi_total_hh,...,median_value_owner_occupied_units,households,hh_families,hh_mc_families,hh_mc_with_own_children_under_18,hh_sp_families,hh_sp_with_own_children_under_18,hh_non_families,aland_sq_mi,geoid
4,New York,New York,NaN,NaN,2023,19571216,415.3,84578,125909,7668956,...,403000,7668956,4739175,3290138,1270627,1449037,617727,2929781,47123.149121,0400000US36
5,06390,New York,Suffolk,Southold,2023,53,13.1,51964,85400,17,...,-1,17,10,10,4,0,0,7,4.046667,860Z200US06390
6,10001,New York,New York,New York,2023,29079,46599,123393,205444,15097,...,793000,15097,4715,3291,1078,1424,664,10382,0.624023,860Z200US10001
7,10002,New York,NaN,NaN,2023,75517,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10003,New York,NaN,NaN,2023,53825,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10004,New York,NaN,NaN,2023,3875,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#clean the population by zip code
clean_df = pop_dir[['Geography', 'Unnamed: 5']].copy()

In [6]:
clean_df.head(10)

,Geography,Unnamed: 5
0,NaN,NaN
1,NaN,Population
2,NaN,NaN
3,name,population
4,New York,19571216
5,06390,53
6,10001,29079
7,10002,75517
8,10003,53825
9,10004,3875


In [7]:
# Reset the index to make row manipulation easier
clean_df = clean_df.reset_index(drop=True)

In [8]:
# Use row 3 as the header and drop the first 4 rows
clean_df.columns = clean_df.iloc[3]
clean_df = clean_df.iloc[4:].reset_index(drop=True)

In [9]:
# Convert the population column to numeric values
clean_df['population'] = pd.to_numeric(clean_df['population'], errors='coerce')


In [10]:
# Create a new column to identify if the entry is a ZIP code or place name
clean_df['is_zipcode'] = clean_df['name'].str.match(r'^\d+$')

In [11]:
# Rename the columns for clarity
clean_df = clean_df.rename(columns={'name': 'geography'})

# Display the cleaned dataframe
print(clean_df)

3    geography  population  is_zipcode
0     New York  19571216.0       False
1        06390        53.0        True
2        10001     29079.0        True
3        10002     75517.0        True
4        10003     53825.0        True
...        ...         ...         ...
1822     14898      1501.0        True
1823     14901     14430.0        True
1824     14903      7140.0        True
1825     14904     14276.0        True
1826     14905      8976.0        True

[1827 rows x 3 columns]


In [12]:
clean_df.to_csv("clean_data/population_clean.csv", index=False)

# Income

In [13]:
income_df=pd.read_csv("data/Income/ACSST5Y2023.S1903-Data.csv")

In [14]:
income_df.head()

,GEO_ID,NAME,S1903_C01_001E,S1903_C01_001M,S1903_C01_002E,S1903_C01_002M,S1903_C01_003E,S1903_C01_003M,S1903_C01_004E,S1903_C01_004M,...,S1903_C03_036M,S1903_C03_037E,S1903_C03_037M,S1903_C03_038E,S1903_C03_038M,S1903_C03_039E,S1903_C03_039M,S1903_C03_040E,S1903_C03_040M,Unnamed: 242
0,Geography,Geographic Area Name,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND...,Margin of Error!!Number!!HOUSEHOLD INCOME BY R...,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND...,Margin of Error!!Number!!HOUSEHOLD INCOME BY R...,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND...,Margin of Error!!Number!!HOUSEHOLD INCOME BY R...,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND...,Margin of Error!!Number!!HOUSEHOLD INCOME BY R...,...,Margin of Error!!Median income (dollars)!!NONF...,Estimate!!Median income (dollars)!!NONFAMILY H...,Margin of Error!!Median income (dollars)!!NONF...,Estimate!!Median income (dollars)!!NONFAMILY H...,Margin of Error!!Median income (dollars)!!NONF...,Estimate!!Median income (dollars)!!NONFAMILY H...,Margin of Error!!Median income (dollars)!!NONF...,Estimate!!Median income (dollars)!!NONFAMILY H...,Margin of Error!!Median income (dollars)!!NONF...,NaN
1,860Z200US06390,ZCTA5 06390,17,13,17,13,0,13,0,13,...,**,-,**,-,**,-,**,-,**,NaN
2,860Z200US10001,ZCTA5 10001,15097,796,8810,675,1441,359,32,40,...,10860,"250,000+",***,156563,34367,107149,24995,234638,53290,NaN
3,860Z200US10002,ZCTA5 10002,35771,1265,12313,895,2821,474,92,70,...,5172,128973,34470,45394,9075,36743,5411,198188,29003,NaN
4,860Z200US10003,ZCTA5 10003,25080,1152,19281,1093,356,177,15,24,...,15734,163808,28768,145063,30998,112855,19517,"250,000+",***,NaN


In [15]:
# Check column names
print(income_df.columns.tolist())

['GEO_ID', 'NAME', 'S1903_C01_001E', 'S1903_C01_001M', 'S1903_C01_002E', 'S1903_C01_002M', 'S1903_C01_003E', 'S1903_C01_003M', 'S1903_C01_004E', 'S1903_C01_004M', 'S1903_C01_005E', 'S1903_C01_005M', 'S1903_C01_006E', 'S1903_C01_006M', 'S1903_C01_007E', 'S1903_C01_007M', 'S1903_C01_008E', 'S1903_C01_008M', 'S1903_C01_009E', 'S1903_C01_009M', 'S1903_C01_010E', 'S1903_C01_010M', 'S1903_C01_011E', 'S1903_C01_011M', 'S1903_C01_012E', 'S1903_C01_012M', 'S1903_C01_013E', 'S1903_C01_013M', 'S1903_C01_014E', 'S1903_C01_014M', 'S1903_C01_015E', 'S1903_C01_015M', 'S1903_C01_016E', 'S1903_C01_016M', 'S1903_C01_017E', 'S1903_C01_017M', 'S1903_C01_018E', 'S1903_C01_018M', 'S1903_C01_019E', 'S1903_C01_019M', 'S1903_C01_020E', 'S1903_C01_020M', 'S1903_C01_021E', 'S1903_C01_021M', 'S1903_C01_022E', 'S1903_C01_022M', 'S1903_C01_023E', 'S1903_C01_023M', 'S1903_C01_024E', 'S1903_C01_024M', 'S1903_C01_025E', 'S1903_C01_025M', 'S1903_C01_026E', 'S1903_C01_026M', 'S1903_C01_027E', 'S1903_C01_027M', 'S1903_C0

In [16]:
if 'GEO_ID' in income_df.columns:
    # Skip header row if it exists
    if income_df.iloc[0]['GEO_ID'].startswith('Geographic'):
        income_df = income_df.iloc[1:].reset_index(drop=True)


In [17]:
required_cols = ['GEO_ID', 'NAME']
if 'S1903_C01_001E' in income_df.columns:
    required_cols.append('S1903_C01_001E')  # Median household income
    
clean_df = income_df[required_cols].copy()

In [18]:
if 'NAME' in clean_df.columns:
    clean_df['zip_code'] = clean_df['NAME'].str.extract(r'ZCTA5 (\d{5})')


In [19]:
if 'S1903_C01_001E' in clean_df.columns:
    clean_df.rename(columns={'S1903_C01_001E': 'Median_Household_Income'}, inplace=True)
    clean_df['Median_Household_Income'] = pd.to_numeric(clean_df['Median_Household_Income'], errors='coerce')


In [20]:
# Final cleanup - select only needed columns
final_columns = ['zip_code', 'Median_Household_Income']

if all(col in clean_df.columns for col in final_columns):
    clean_income_df = clean_df[final_columns].copy()
    

In [21]:
clean_income_df.to_csv('clean_data/clean_income_data.csv', index=False)

print(f"Successfully cleaned income data. Shape: {clean_income_df.shape}")
print("\nSample data:")
print(clean_income_df.head())

Successfully cleaned income data. Shape: (1825, 2)

Sample data:
  zip_code  Median_Household_Income
0      NaN                      NaN
1    06390                     17.0
2    10001                  15097.0
3    10002                  35771.0
4    10003                  25080.0


In [22]:
print("\nSummary statistics:")
print(clean_income_df.info())


Summary statistics:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   zip_code                 1824 non-null   object 
 1   Median_Household_Income  1824 non-null   float64
dtypes: float64(1), object(1)
memory usage: 28.6+ KB
None


# Demographic

In [23]:
# now we will process the demographic data
demographics_df = pd.read_csv("data/Demographic/ACSDP5Y2023.DP05-Data.csv")

In [24]:
demographics_df.head()

,GEO_ID,NAME,DP05_0001E,DP05_0001M,DP05_0002E,DP05_0002M,DP05_0003E,DP05_0003M,DP05_0004E,DP05_0004M,...,DP05_0090PM,DP05_0091PE,DP05_0091PM,DP05_0092PE,DP05_0092PM,DP05_0093PE,DP05_0093PM,DP05_0094PE,DP05_0094PM,Unnamed: 378
0,Geography,Geographic Area Name,Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Total population!!Male,Margin of Error!!SEX AND AGE!!Total population...,Estimate!!SEX AND AGE!!Total population!!Female,Margin of Error!!SEX AND AGE!!Total population...,Estimate!!SEX AND AGE!!Total population!!Sex r...,Margin of Error!!SEX AND AGE!!Total population...,...,Percent Margin of Error!!HISPANIC OR LATINO AN...,Percent!!Total housing units,Percent Margin of Error!!Total housing units,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...","Percent Margin of Error!!CITIZEN, VOTING AGE P...","Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...","Percent Margin of Error!!CITIZEN, VOTING AGE P...","Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...","Percent Margin of Error!!CITIZEN, VOTING AGE P...",NaN
1,860Z200US06390,ZCTA5 06390,53,34,37,28,16,14,231.3,302.7,...,10.5,(X),(X),34,(X),58.8,33.7,41.2,33.7,NaN
2,860Z200US10001,ZCTA5 10001,29079,1790,14069,1231,15010,1357,93.7,12.2,...,1.1,(X),(X),21633,(X),47.5,3.6,52.5,3.6,NaN
3,860Z200US10002,ZCTA5 10002,75517,3196,38110,2445,37407,1962,101.9,8.3,...,0.6,(X),(X),54901,(X),48.9,2.3,51.1,2.3,NaN
4,860Z200US10003,ZCTA5 10003,53825,2771,26407,1627,27418,2112,96.3,9.1,...,0.8,(X),(X),44743,(X),49.1,2.2,50.9,2.2,NaN


In [25]:
# Check column names
print(demographics_df.columns.tolist())

['GEO_ID', 'NAME', 'DP05_0001E', 'DP05_0001M', 'DP05_0002E', 'DP05_0002M', 'DP05_0003E', 'DP05_0003M', 'DP05_0004E', 'DP05_0004M', 'DP05_0005E', 'DP05_0005M', 'DP05_0006E', 'DP05_0006M', 'DP05_0007E', 'DP05_0007M', 'DP05_0008E', 'DP05_0008M', 'DP05_0009E', 'DP05_0009M', 'DP05_0010E', 'DP05_0010M', 'DP05_0011E', 'DP05_0011M', 'DP05_0012E', 'DP05_0012M', 'DP05_0013E', 'DP05_0013M', 'DP05_0014E', 'DP05_0014M', 'DP05_0015E', 'DP05_0015M', 'DP05_0016E', 'DP05_0016M', 'DP05_0017E', 'DP05_0017M', 'DP05_0018E', 'DP05_0018M', 'DP05_0019E', 'DP05_0019M', 'DP05_0020E', 'DP05_0020M', 'DP05_0021E', 'DP05_0021M', 'DP05_0022E', 'DP05_0022M', 'DP05_0023E', 'DP05_0023M', 'DP05_0024E', 'DP05_0024M', 'DP05_0025E', 'DP05_0025M', 'DP05_0026E', 'DP05_0026M', 'DP05_0027E', 'DP05_0027M', 'DP05_0028E', 'DP05_0028M', 'DP05_0029E', 'DP05_0029M', 'DP05_0030E', 'DP05_0030M', 'DP05_0031E', 'DP05_0031M', 'DP05_0032E', 'DP05_0032M', 'DP05_0033E', 'DP05_0033M', 'DP05_0034E', 'DP05_0034M', 'DP05_0035E', 'DP05_0035M', '

In [26]:
# we want DP05_0001E -> Total Population
#Name -> Zipcode
#DP05_0002E-> male population
#DP05_0003E -> female population
#DP05_0005E -> under 5 years
#DP05_0006E -> 5 to 9 years
#DP05_0007E -> 10 to 14 years
#DP05_0008E -> 15 to 19 years
#DP05_0009E -> 20 to 24 years
#DP05_0010E -> 25 to 34 years
#DP05_0011E -> 35 to 44 years
#DP05_0012E -> 45 to 54 years
#DP05_0013E -> 55 to 64 years
#DP05_0014E -> 65 years and over

#DP05_0037E -> White alone
#DP05_0039E -> American Indian and Alaska Native alone
#DP05_0047E -> Asian alone


In [27]:
# Select relevant columns
selected_columns = [
    'NAME',
    'DP05_0001E',  # Total Population
    'DP05_0002E',  # Male population
    'DP05_0003E',  # Female population
    'DP05_0005E',  # Under 5 years
    'DP05_0006E',  # 5 to 9 years
    'DP05_0007E',  # 10 to 14 years
    'DP05_0008E',  # 15 to 19 years
    'DP05_0009E',  # 20 to 24 years
    'DP05_0010E',  # 25 to 34 years
    'DP05_0011E',  # 35 to 44 years
    'DP05_0012E',  # 45 to 54 years
    'DP05_0013E',  # 55 to 64 years
    'DP05_0014E',  # 65 years and over
    'DP05_0037E',  # White alone
    'DP05_0039E',  # American Indian and Alaska Native alone
    'DP05_0047E'   # Asian alone
]


In [28]:
# Filter the demographics dataframe
clean_demographics = demographics_df[selected_columns].copy()

# Extract ZIP codes from NAME column
clean_demographics['zip_code'] = clean_demographics['NAME'].str.extract(r'ZCTA5 (\d{5})')

# Convert columns to numeric
for col in selected_columns[1:]:  # Skip 'NAME'
    clean_demographics[col] = pd.to_numeric(clean_demographics[col], errors='coerce')
    

In [29]:
# Rename columns for clarity
column_mapping = {
    'DP05_0001E': 'total_population',
    'DP05_0002E': 'male_population',
    'DP05_0003E': 'female_population',
    'DP05_0005E': 'age_under_5',
    'DP05_0006E': 'age_5_to_9',
    'DP05_0007E': 'age_10_to_14',
    'DP05_0008E': 'age_15_to_19',
    'DP05_0009E': 'age_20_to_24',
    'DP05_0010E': 'age_25_to_34',
    'DP05_0011E': 'age_35_to_44',
    'DP05_0012E': 'age_45_to_54',
    'DP05_0013E': 'age_55_to_64',
    'DP05_0014E': 'age_65_plus',
    'DP05_0037E': 'race_white',
    'DP05_0039E': 'race_native_american',
    'DP05_0047E': 'race_asian'
}

In [30]:
clean_demographics.rename(columns=column_mapping, inplace=True)


In [31]:
# Create aggregate age groups that might be useful for analysis
clean_demographics['age_under_18'] = (
    clean_demographics['age_under_5'] + 
    clean_demographics['age_5_to_9'] + 
    clean_demographics['age_10_to_14'] + 
    clean_demographics['age_15_to_19'] * 0.8  # Approximately 80% of 15-19 are under 18
)

clean_demographics['age_18_to_44'] = (
    clean_demographics['age_15_to_19'] * 0.2 +  # Approximately 20% of 15-19 are 18-19
    clean_demographics['age_20_to_24'] + 
    clean_demographics['age_25_to_34'] + 
    clean_demographics['age_35_to_44']
)

clean_demographics['age_45_plus'] = (
    clean_demographics['age_45_to_54'] + 
    clean_demographics['age_55_to_64'] + 
    clean_demographics['age_65_plus']
)

# Select final columns for the clean dataset
final_columns = ['zip_code', 'total_population', 'male_population', 'female_population',
                'age_under_18', 'age_18_to_44', 'age_45_plus', 
                'race_white', 'race_native_american', 'race_asian']

In [32]:
# Create final dataframe
final_demographics = clean_demographics[final_columns].copy()
final_demographics.dropna(subset=['zip_code'], inplace=True)
# Save the cleaned demographics data
final_demographics.to_csv('clean_data/clean_demographics_data.csv', index=False)

# Education

In [33]:
education_df=pd.read_csv("data/education/ACSST5Y2023.S1501-Data.csv")

/var/folders/wg/h6z2ybxs0bnfsz9zc363lx_w0000gn/T/ipykernel_57071/793348813.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,54

In [34]:
education_df.head()

,GEO_ID,NAME,S1501_C01_001E,S1501_C01_001M,S1501_C01_002E,S1501_C01_002M,S1501_C01_003E,S1501_C01_003M,S1501_C01_004E,S1501_C01_004M,...,S1501_C06_060M,S1501_C06_061E,S1501_C06_061M,S1501_C06_062E,S1501_C06_062M,S1501_C06_063E,S1501_C06_063M,S1501_C06_064E,S1501_C06_064M,Unnamed: 770
0,Geography,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,NaN
1,860Z200US06390,ZCTA5 06390,7,10,0,13,0,13,0,13,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
2,860Z200US10001,ZCTA5 10001,3033,389,37,53,1121,248,860,253,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
3,860Z200US10002,ZCTA5 10002,5978,783,533,277,951,302,2655,507,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
4,860Z200US10003,ZCTA5 10003,13489,798,93,71,4002,788,6132,726,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN


In [35]:
# Skip header row if present
if 'GEO_ID' in education_df.columns and education_df.iloc[0]['GEO_ID'].startswith('Geographic'):
    education_df = education_df.iloc[1:].reset_index(drop=True)

In [36]:
education_df.head()

,GEO_ID,NAME,S1501_C01_001E,S1501_C01_001M,S1501_C01_002E,S1501_C01_002M,S1501_C01_003E,S1501_C01_003M,S1501_C01_004E,S1501_C01_004M,...,S1501_C06_060M,S1501_C06_061E,S1501_C06_061M,S1501_C06_062E,S1501_C06_062M,S1501_C06_063E,S1501_C06_063M,S1501_C06_064E,S1501_C06_064M,Unnamed: 770
0,Geography,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,NaN
1,860Z200US06390,ZCTA5 06390,7,10,0,13,0,13,0,13,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
2,860Z200US10001,ZCTA5 10001,3033,389,37,53,1121,248,860,253,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
3,860Z200US10002,ZCTA5 10002,5978,783,533,277,951,302,2655,507,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
4,860Z200US10003,ZCTA5 10003,13489,798,93,71,4002,788,6132,726,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN


In [37]:
# Education column mapping
education_columns = {
    'NAME': 'name',                      # Geographic name (includes ZIP code)
    'S1501_C01_006E': 'pop_25_over',     # Population 25 years and over 
    'S1501_C02_015E': 'pct_bachelors_plus_25_over',  # % Bachelor's degree or higher (25+)
    'S1501_C01_001E': 'pop_18_24',       # Population 18 to 24 years
    'S1501_C02_005E': 'pct_bachelor_and_over_18_24',  # % Bachelor's degree or higher (18-24) 
}

In [38]:
# Select and rename columns
clean_education = education_df[education_columns.keys()].copy()
clean_education.rename(columns=education_columns, inplace=True)


In [39]:
# Extract ZIP codes
clean_education['zip_code'] = clean_education['name'].str.extract(r'ZCTA5 (\d{5})')


In [40]:
# Convert columns to numeric
numeric_cols = [col for col in clean_education.columns if col not in ['name', 'zip_code']]
for col in numeric_cols:
    clean_education[col] = pd.to_numeric(clean_education[col], errors='coerce')

In [41]:
clean_education.head()

,name,pop_25_over,pct_bachelors_plus_25_over,pop_18_24,pct_bachelor_and_over_18_24,zip_code
0,Geographic Area Name,NaN,NaN,NaN,NaN,NaN
1,ZCTA5 06390,38.0,39.5,7.0,100.0,06390
2,ZCTA5 10001,22724.0,71.5,3033.0,33.5,10001
3,ZCTA5 10002,59418.0,39.0,5978.0,30.8,10002
4,ZCTA5 10003,36544.0,82.2,13489.0,24.2,10003


In [42]:
# Calculate raw counts from percentages
clean_education['count_bachelors_plus_25_over'] = (
    clean_education['pop_25_over'] * clean_education['pct_bachelors_plus_25_over'] / 100
).round().astype('Int64')  # Using Int64 to handle NaN values


In [43]:
clean_education['count_bachelors_plus_18_24'] = (
    clean_education['pop_18_24'] * clean_education['pct_bachelor_and_over_18_24'] / 100
).round().astype('Int64')

In [44]:
# Final columns selection
final_cols = [
    'zip_code', 
    'pop_25_over', 'pct_bachelors_plus_25_over', 'count_bachelors_plus_25_over',
    'pop_18_24', 'pct_bachelor_and_over_18_24', 'count_bachelors_plus_18_24'
]

In [45]:
final_education = clean_education[final_cols].copy()

In [46]:
final_education.head()
#dropNA
final_education.dropna(subset=['zip_code'], inplace=True)
# Save the cleaned education data
final_education.to_csv('clean_data/clean_education_data.csv', index=False)

# Retail_Food_Store

In [47]:
bus_sec=pd.read_csv("data/Retail_Food_Stores_20250408.csv")

In [48]:
bus_sec.head()

,County,License Number,Operation Type,Establishment Type,Entity Name,DBA Name,Street Number,Street Name,Address Line 2,Address Line 3,City,State,Zip Code,Square Footage,Georeference
0,SUFFOLK,763163,Store,A,HEALTHY MEALS DIRECT LLC,HEALTHY MEALS DIRECT,1866,DEER PARK AVE,NaN,NaN,DEER PARK,NY,11729,NaN,POINT (-73.32901606 40.7599309)
1,TIOGA,763162,Store,A,ALDI INC,ALDI #62,1150,STATE ROUTE 17C,NaN,NaN,OWEGO,NY,13827,NaN,POINT (-76.231859408 42.096557734)
2,WESTCHESTER,763161,Store,A,WALGREEN EASTERN CO INC,WALGREENS #21443,3320,CROMPOND RD,NaN,NaN,YORKTOWN HEIGHTS,NY,10598,NaN,POINT (-73.830051866 41.291524945)
3,SUFFOLK,763134,Store,A,SANJHA BAZAAR LLC,SANJHA BAZAAR,2160,JERICHO TURNPIKE,NaN,NaN,COMMACK,NY,11725,NaN,POINT (-73.284552495 40.842579189)
4,KINGS,763133,Store,A,SKILLMART INC,SKILLMART,1010,BEDFORD AVE,NaN,NaN,BROOKLYN,NY,11205,NaN,POINT (-73.955486796 40.690346184)


In [94]:
bus_sec['County'].unique()

array(['SUFFOLK', 'TIOGA', 'WESTCHESTER', 'KINGS', 'BRONX', 'ONEIDA',
       'NASSAU', 'QUEENS', 'ALBANY', 'ROCKLAND', 'NEW YORK', 'DUTCHESS',
       'ESSEX', 'BROOME', 'RICHMOND', 'TOMPKINS', 'ORANGE', 'SULLIVAN',
       'OSWEGO', 'DELAWARE', 'MONROE', 'NIAGARA', 'STEUBEN', 'LEWIS',
       'ONONDAGA', 'ERIE', 'CATTARAUGUS', 'MADISON', 'WASHINGTON',
       'CHENANGO', 'COLUMBIA', 'CHAUTAUQUA', 'SENECA', 'FULTON',
       'SCHENECTADY', 'RENSSELAER', 'CHEMUNG', 'OTSEGO', 'MONTGOMERY',
       'SARATOGA', 'JEFFERSON', 'YATES', 'GREENE', 'WAYNE', 'ALLEGANY',
       'SCHUYLER', 'GENESEE', 'ULSTER', 'CLINTON', 'HERKIMER', 'HAMILTON',
       'WARREN', 'WYOMING', 'LIVINGSTON', 'FRANKLIN', 'ONTARIO',
       'CORTLAND', 'CAYUGA', 'ORLEANS', 'PUTNAM', 'ST. LAWRENCE',
       'SCHOHARIE', nan], dtype=object)

In [ ]:
nyc_counties = ['BRONX', 'KINGS', 'NEW YORK', 'QUEENS', 'RICHMOND']
nyc_stores = bus_sec[bus_sec['County'].str.upper().isin(nyc_counties)].copy()
print(f"Total stores in dataset: {len(bus_sec)}")
print(f"Stores in NYC: {len(nyc_stores)}")



Total stores in dataset: 24221
Stores in NYC: 11352


In [50]:
nyc_stores.head()

,County,License Number,Operation Type,Establishment Type,Entity Name,DBA Name,Street Number,Street Name,Address Line 2,Address Line 3,City,State,Zip Code,Square Footage,Georeference
4,KINGS,763133,Store,A,SKILLMART INC,SKILLMART,1010,BEDFORD AVE,NaN,NaN,BROOKLYN,NY,11205,NaN,POINT (-73.955486796 40.690346184)
5,BRONX,763132,Store,A,ARDENT PHARMACY INC,BENJAMINS PHARMACY & SURGICAL,987-989,ALLERTON AVE,NaN,NaN,BRONX,NY,10469,NaN,POINT (-73.858823252 40.865713996)
9,BRONX,763012,Store,A,BALDE FAMILY GENERAL MERCHANDISING LLC,BALDE FAMILY GENERAL MERCHANDISING,756,BURKE AVENUE,NaN,NaN,BRONX,NY,10467,NaN,POINT (-73.864832322 40.871259212)
10,BRONX,763011,Store,A,ESTRELLA MEATS FRUITS & GROCERY 2 CORP,ESTRELLA MEATS FRUITS & GROCERY 2,350,E GUN HILL RD,NaN,NaN,BRONX,NY,10467,NaN,POINT (-73.872836453 40.878631389)
11,QUEENS,762997,Store,A,HUA-TE NY INC,HUA-TE NY,42-14,MAIN ST,NaN,NaN,FLUSHING,NY,11355,NaN,POINT (-73.828313124 40.754958503)


In [95]:
nyc_stores['Zip Code'].value_counts()

Zip Code
11220    221
11207    198
10467    179
11226    178
11368    176
        ... 
10120      1
11697      1
10069      1
10118      1
10110      1
Name: count, Length: 187, dtype: int64

In [51]:
# Select the columns you want to keep
store_details = nyc_stores[['DBA Name', 'Zip Code', 'Square Footage', 'Operation Type', 'Georeference']].copy()

# Clean ZIP codes
store_details['Zip_Code_Clean'] = store_details['Zip Code'].astype(str).str.split('-').str[0].str.strip()

# Make column names more consistent
store_details = store_details.rename(columns={
    'DBA Name': 'store_name',
    'Square Footage': 'square_footage',
    'Operation Type': 'operation_type',
    'Georeference': 'georeference'
})

# Handle any missing values
store_details['square_footage'] = pd.to_numeric(store_details['square_footage'], errors='coerce')
store_details_columns=['store_name','square_footage','operation_type','georeference','Zip_Code_Clean']

store_details = store_details[store_details_columns].copy()
store_details.dropna(subset=['Zip_Code_Clean'], inplace=True)
# Save the cleaned store data
store_details.to_csv('clean_data/store_details.csv', index=False)
print(f"Successfully cleaned store data. Shape: {store_details.shape}")

Successfully cleaned store data. Shape: (11352, 5)


In [52]:
nyc_stores['Zip_Code_Clean'] = nyc_stores['Zip Code'].astype(str).str.split('-').str[0].str.strip()


In [53]:
nyc_stores['Store_Category'] = nyc_stores['Establishment Type'] + '-' + nyc_stores['Operation Type']


In [54]:
from shapely import wkt

In [55]:
nyc_stores['Coordinates'] = nyc_stores['Georeference'].apply(
    lambda x: wkt.loads(x) if isinstance(x, str) else None
)

In [56]:
gdf_stores = gpd.GeoDataFrame(
    nyc_stores, 
    geometry='Coordinates',
    crs="EPSG:4326"
)


In [57]:
zip_store_counts = nyc_stores.groupby('Zip_Code_Clean').size().reset_index(name='store_count')


In [58]:
grocery_counts = nyc_stores[nyc_stores['Establishment Type'] == 'A'].groupby('Zip_Code_Clean').size().reset_index(name='grocery_count')


In [59]:
store_counts_by_zip = pd.merge(zip_store_counts, grocery_counts, on='Zip_Code_Clean', how='left')
store_counts_by_zip['grocery_count'] = store_counts_by_zip['grocery_count'].fillna(0).astype(int)


In [60]:
specialty_keywords = ['ORGANIC', 'NATURAL', 'HEALTH', 'SPECIALTY', 'GOURMET', 'MARKET']
nyc_stores['is_specialty'] = nyc_stores['DBA Name'].str.upper().apply(
    lambda x: any(keyword in x for keyword in specialty_keywords) if isinstance(x, str) else False
)

In [61]:
specialty_counts = nyc_stores[nyc_stores['is_specialty']].groupby('Zip_Code_Clean').size().reset_index(name='specialty_count')
store_counts_by_zip = pd.merge(store_counts_by_zip, specialty_counts, on='Zip_Code_Clean', how='left')
store_counts_by_zip['specialty_count'] = store_counts_by_zip['specialty_count'].fillna(0).astype(int)


In [62]:
store_counts_by_zip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Zip_Code_Clean   187 non-null    object
 1   store_count      187 non-null    int64 
 2   grocery_count    187 non-null    int64 
 3   specialty_count  187 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.0+ KB


In [63]:
store_counts_by_zip.to_csv('clean_data/store_counts_by_zip.csv', index=False)
print(f"Successfully cleaned store counts data. Shape: {store_counts_by_zip.shape}")

Successfully cleaned store counts data. Shape: (187, 4)


In [64]:
# Group by store name to find chains
chain_stores = bus_sec.groupby('DBA Name').agg({
    'License Number': 'count',  # Count locations
    'County': pd.Series.nunique,  # Count unique counties (to see cross-county chains)
    'Zip Code': lambda x: pd.Series.nunique(pd.Series(x.astype(str).str.split('-').str[0])),  # Unique ZIP codes
    'Square Footage': 'mean'  # Average store size
}).reset_index()

# Define chains as stores with 3+ locations
chain_stores = chain_stores[chain_stores['License Number'] >= 3].sort_values('License Number', ascending=False)

# Display top chains
print(f"Found {len(chain_stores)} chain stores (3+ locations)")
print("\nTop 10 chain stores by number of locations:")
print(chain_stores.head())

# Save chains data
chain_stores.to_csv('clean_data/nyc_chain_stores.csv', index=False)


Found 226 chain stores (3+ locations)

Top 10 chain stores by number of locations:
                  DBA Name  License Number  County  Zip Code  Square Footage
7860   EDIBLE ARRANGEMENTS              48      20        48     1425.365854
11987             KEY FOOD              45       7        41    12451.351351
4273                C TOWN              40       7        37    11378.378378
14585           MOBIL MART              33      12        28     1537.757576
11065    IDEAL FOOD BASKET              26       6        21    14700.000000


In [96]:
len(chain_stores)

226

In [65]:
# Count chains by ZIP code
chains_by_zip = pd.merge(
    bus_sec,
    chain_stores[['DBA Name']],
    on='DBA Name',
    how='inner'
)

# Clean ZIP codes
chains_by_zip['Zip_Code_Clean'] = chains_by_zip['Zip Code'].astype(str).str.split('-').str[0].str.strip()

# Count chain stores per ZIP
chain_count_by_zip = chains_by_zip.groupby('Zip_Code_Clean').size().reset_index(name='chain_store_count')

# Merge with our existing data
store_counts_by_zip = pd.merge(store_counts_by_zip, chain_count_by_zip, on='Zip_Code_Clean', how='left')
store_counts_by_zip['chain_store_count'] = store_counts_by_zip['chain_store_count'].fillna(0).astype(int)

# Calculate percentage of stores that are chains
store_counts_by_zip['chain_store_pct'] = (store_counts_by_zip['chain_store_count'] / 
                                          store_counts_by_zip['store_count'] * 100).round(1)

# Show updated data
print("\nZIP codes with highest chain store presence:")
print(store_counts_by_zip.sort_values('chain_store_count', ascending=False).head())


ZIP codes with highest chain store presence:
    Zip_Code_Clean  store_count  grocery_count  specialty_count  \
107          11211          150             16               46   
103          11207          198             20               48   
108          11212          153             28               39   
149          11368          176             29               33   
67           10453          118             14               27   

     chain_store_count  chain_store_pct  
107                 15             10.0  
103                 14              7.1  
108                 13              8.5  
149                 12              6.8  
67                  12             10.2  


# Remote Work Proportion

In [66]:
df=pd.read_csv("data/remote_work_proportion.csv")

In [67]:
df.columns.to_list()

['Label (Grouping)',
 'St. Lawrence County PUMA; New York!!Estimate',
 'St. Lawrence County PUMA; New York!!Percent',
 'Clinton, Franklin, Essex & Hamilton Counties PUMA; New York!!Estimate',
 'Clinton, Franklin, Essex & Hamilton Counties PUMA; New York!!Percent',
 'Warren & Washington Counties PUMA; New York!!Estimate',
 'Warren & Washington Counties PUMA; New York!!Percent',
 'Herkimer (North & Central) & Oneida (Outer) Counties PUMA; New York!!Estimate',
 'Herkimer (North & Central) & Oneida (Outer) Counties PUMA; New York!!Percent',
 'Oneida County (Central)--Greater Utica & Rome Cities PUMA; New York!!Estimate',
 'Oneida County (Central)--Greater Utica & Rome Cities PUMA; New York!!Percent',
 'Otsego, Schoharie, Oneida (South) & Herkimer (South) Counties PUMA; New York!!Estimate',
 'Otsego, Schoharie, Oneida (South) & Herkimer (South) Counties PUMA; New York!!Percent',
 'Jefferson & Lewis Counties PUMA; New York!!Estimate',
 'Jefferson & Lewis Counties PUMA; New York!!Percent',
 '

In [68]:
district_columns = [col for col in df.columns if 'NYC-' in col and 'Estimate' in col]

In [69]:
population_mask = df['Label (Grouping)'].str.contains('Population 16 years and over', na=False)
workers_mask = df['Label (Grouping)'].str.contains('Workers 16 years and over', na=False)
    

In [70]:
population_data = df[population_mask][district_columns].iloc[0]
workers_data = df[workers_mask][district_columns].iloc[0]
print("Population Data:")
print(population_data)
print("\nWorkers Data:")
print(workers_data)

Population Data:
NYC-Manhattan Community District 3--Lower East Side & Chinatown PUMA; New York!!Estimate                      133,718
NYC-Manhattan Community District 4--Chelsea & Hell's Kitchen PUMA; New York!!Estimate                         112,097
NYC-Manhattan Community District 7--Upper West Side PUMA; New York!!Estimate                                  185,973
NYC-Manhattan Community District 8--Upper East Side & Roosevelt Island PUMA; New York!!Estimate               176,157
NYC-Manhattan Community District 9--Morningside Heights & Hamilton Heights PUMA; New York!!Estimate            99,184
NYC-Manhattan Community District 10--Harlem PUMA; New York!!Estimate                                          113,963
NYC-Manhattan Community District 11--East Harlem PUMA; New York!!Estimate                                     102,901
NYC-Manhattan Community District 12--Washington Heights & Inwood PUMA; New York!!Estimate                     154,536
NYC-Manhattan Community Districts 1 & 2

In [71]:
work_df = pd.DataFrame({
    'total_workers': population_data,
    'commuting_workers': workers_data
})

In [72]:
work_df.head()

,total_workers,commuting_workers
NYC-Manhattan Community District 3--Lower East Side & Chinatown PUMA; New York!!Estimate,"133,718","71,201"
NYC-Manhattan Community District 4--Chelsea & Hell's Kitchen PUMA; New York!!Estimate,"112,097","75,883"
NYC-Manhattan Community District 7--Upper West Side PUMA; New York!!Estimate,"185,973","118,279"
NYC-Manhattan Community District 8--Upper East Side & Roosevelt Island PUMA; New York!!Estimate,"176,157","117,825"
NYC-Manhattan Community District 9--Morningside Heights & Hamilton Heights PUMA; New York!!Estimate,"99,184","49,332"


In [73]:
# First, remove commas from the strings
work_df['total_workers'] = work_df['total_workers'].astype(str).str.replace(',', '')
work_df['commuting_workers'] = work_df['commuting_workers'].astype(str).str.replace(',', '')

# Then convert to numeric
work_df['total_workers'] = pd.to_numeric(work_df['total_workers'], errors='coerce')
work_df['commuting_workers'] = pd.to_numeric(work_df['commuting_workers'], errors='coerce')

# Check the data types
print(work_df.dtypes)

# Verify the conversion worked by showing a few rows
print("\nConverted data:")
print(work_df.head())

total_workers        int64
commuting_workers    int64
dtype: object

Converted data:
                                                    total_workers  \
NYC-Manhattan Community District 3--Lower East ...         133718   
NYC-Manhattan Community District 4--Chelsea & H...         112097   
NYC-Manhattan Community District 7--Upper West ...         185973   
NYC-Manhattan Community District 8--Upper East ...         176157   
NYC-Manhattan Community District 9--Morningside...          99184   

                                                    commuting_workers  
NYC-Manhattan Community District 3--Lower East ...              71201  
NYC-Manhattan Community District 4--Chelsea & H...              75883  
NYC-Manhattan Community District 7--Upper West ...             118279  
NYC-Manhattan Community District 8--Upper East ...             117825  
NYC-Manhattan Community District 9--Morningside...              49332  


In [74]:
# Calculate remote workers (people who don't commute)
work_df['remote_workers'] = work_df['total_workers'] - work_df['commuting_workers']


In [75]:
# Calculate remote work percentage
work_df['remote_pct'] = (work_df['remote_workers'] / work_df['total_workers'] * 100).round(1)

# Replace any NaN or infinity values with 0 (in case of divisions by zero)
work_df = work_df.replace([np.inf, -np.inf], np.nan).fillna(0)

# Sort by remote work percentage to see which districts have highest remote work
sorted_df = work_df.sort_values('remote_pct', ascending=False)


In [76]:
sorted_df.head()

,total_workers,commuting_workers,remote_workers,remote_pct
NYC-Brooklyn Community District 13--Coney Island & Brighton Beach PUMA; New York!!Estimate,89320,38517,50803,56.9
"NYC-Bronx Community Districts 1 & 2--Melrose, Mott Haven, Longwood, & Hunts Point PUMA; New York!!Estimate",108001,49035,58966,54.6
"NYC-Bronx Community Districts 3 & 6--Morrisania, Tremont, Belmont, & West Farms PUMA; New York!!Estimate",124659,58298,66361,53.2
NYC-Bronx Community District 5--Morris Heights & Mount Hope PUMA; New York!!Estimate,87545,43360,44185,50.5
NYC-Manhattan Community District 9--Morningside Heights & Hamilton Heights PUMA; New York!!Estimate,99184,49332,49852,50.3


In [77]:
import re

In [78]:
def simplify_district_name(name):
    # Handle NaN values
    if pd.isna(name):
        return 'Unknown'
    
    name = str(name)
    
    # Use regex to extract the key components
    pattern = r'NYC-([A-Za-z\s]+) Community District (\d+)--([^P]+) PUMA'
    match = re.search(pattern, name)
    
    if match:
        borough = match.group(1).strip()
        district_num = match.group(2)
        neighborhood = match.group(3).strip()
        return f"{borough} CD{district_num} - {neighborhood}"
    else:
        # Fallback for names that don't match the pattern
        # Just remove the trailing parts
        cleaned = re.sub(r'PUMA;.*$', '', name)
        cleaned = re.sub(r'!!.*$', '', cleaned)
        return cleaned.strip()


In [79]:
# Apply to index values
work_df['district_simple'] = work_df.index.map(simplify_district_name)

# Show comparison
comparison = pd.DataFrame({
    'Original Name': work_df.index,
    'Simplified Name': work_df['district_simple']
})
print("Name Comparison (First 5 rows):")
for i in range(min(5, len(comparison))):
    print(f"Original: {comparison['Original Name'][i]}")
    print(f"Simplified: {comparison['Simplified Name'][i]}")
    print("-" * 60)

Name Comparison (First 5 rows):
Original: NYC-Manhattan Community District 3--Lower East Side & Chinatown PUMA; New York!!Estimate
Simplified: Manhattan CD3 - Lower East Side & Chinatown
------------------------------------------------------------
Original: NYC-Manhattan Community District 4--Chelsea & Hell's Kitchen PUMA; New York!!Estimate
Simplified: Manhattan CD4 - Chelsea & Hell's Kitchen
------------------------------------------------------------
Original: NYC-Manhattan Community District 7--Upper West Side PUMA; New York!!Estimate
Simplified: Manhattan CD7 - Upper West Side
------------------------------------------------------------
Original: NYC-Manhattan Community District 8--Upper East Side & Roosevelt Island PUMA; New York!!Estimate
Simplified: Manhattan CD8 - Upper East Side & Roosevelt Island
------------------------------------------------------------
Original: NYC-Manhattan Community District 9--Morningside Heights & Hamilton Heights PUMA; New York!!Estimate
Simplified

/var/folders/wg/h6z2ybxs0bnfsz9zc363lx_w0000gn/T/ipykernel_57071/3283310278.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Original: {comparison['Original Name'][i]}")
/var/folders/wg/h6z2ybxs0bnfsz9zc363lx_w0000gn/T/ipykernel_57071/3283310278.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Simplified: {comparison['Simplified Name'][i]}")


In [80]:
work_df_clean = work_df.copy()


In [82]:
# Create new DataFrame with simplified names as index
work_df_clean = work_df.copy()
work_df_clean.index = work_df['district_simple']
work_df_clean = work_df_clean.drop('district_simple', axis=1)


In [84]:
print("\nClean DataFrame with Simplified District Names:")
print(work_df_clean.info())


Clean DataFrame with Simplified District Names:
<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, Manhattan CD3 - Lower East Side & Chinatown to Staten Island CD3 - South Shore
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   total_workers      55 non-null     int64  
 1   commuting_workers  55 non-null     int64  
 2   remote_workers     55 non-null     int64  
 3   remote_pct         55 non-null     float64
dtypes: float64(1), int64(3)
memory usage: 2.1+ KB
None


In [85]:
# Create a clean DataFrame with simplified district names as a column
clean_work_df = pd.DataFrame({
    'district_simple': work_df.index.map(simplify_district_name),
    'total_workers': work_df['total_workers'],
    'commuting_workers': work_df['commuting_workers'],
    'remote_workers': work_df['remote_workers'],
    'remote_pct': work_df['remote_pct']
})

# Reset index if needed to have numeric indices
clean_work_df = clean_work_df.reset_index(drop=True)

# Display the first few rows
print("Clean DataFrame with All Columns:")
print(clean_work_df.head())

# Save the data
clean_work_df.to_csv('clean_data/nyc_remote_work_by_district.csv', index=False)

# Basic statistics
print("\nRemote Work Statistics:")
print(f"Average remote work percentage: {clean_work_df['remote_pct'].mean():.1f}%")
print(f"Maximum remote work percentage: {clean_work_df['remote_pct'].max():.1f}%")
print(f"Districts with >20% remote work: {len(clean_work_df[clean_work_df['remote_pct'] > 20])}")

Clean DataFrame with All Columns:
                                     district_simple  total_workers  \
0        Manhattan CD3 - Lower East Side & Chinatown         133718   
1           Manhattan CD4 - Chelsea & Hell's Kitchen         112097   
2                    Manhattan CD7 - Upper West Side         185973   
3  Manhattan CD8 - Upper East Side & Roosevelt Is...         176157   
4  Manhattan CD9 - Morningside Heights & Hamilton...          99184   

   commuting_workers  remote_workers  remote_pct  
0              71201           62517        46.8  
1              75883           36214        32.3  
2             118279           67694        36.4  
3             117825           58332        33.1  
4              49332           49852        50.3  

Remote Work Statistics:
Average remote work percentage: 41.9%
Maximum remote work percentage: 56.9%
Districts with >20% remote work: 55


In [86]:
clean_work_df.head()

,district_simple,total_workers,commuting_workers,remote_workers,remote_pct
0,Manhattan CD3 - Lower East Side & Chinatown,133718,71201,62517,46.8
1,Manhattan CD4 - Chelsea & Hell's Kitchen,112097,75883,36214,32.3
2,Manhattan CD7 - Upper West Side,185973,118279,67694,36.4
3,Manhattan CD8 - Upper East Side & Roosevelt Is...,176157,117825,58332,33.1
4,Manhattan CD9 - Morningside Heights & Hamilton...,99184,49332,49852,50.3


In [87]:
clean_work_df.to_csv('clean_data/nyc_remote_work_by_district.csv', index=False)

# storefronts

In [89]:
store_fronts=pd.read_csv("data/Storefronts_Reported_Vacant_or_Not_20250409.csv")

/var/folders/wg/h6z2ybxs0bnfsz9zc363lx_w0000gn/T/ipykernel_57071/146387538.py:1: DtypeWarning: Columns (1,5,6,11,16,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  store_fronts=pd.read_csv("data/Storefronts_Reported_Vacant_or_Not_20250409.csv")


In [90]:
store_fronts.head()

,Filing Due Date,Reporting Year,Borough Block Lot,Property Street Address or Storefront Address,Borough,Zip Code,Sold Date,Vacant on 12/31,Construction Reported,Vacant 6/30 or Date Sold,...,Latitude,Longitude,Lat/Long,Community Board,Council District,Census Tract,BIN,BBL,NTA,NTA Neighborhood
0,06/03/2024,2023,5051430020,3996 AMBOY ROAD,STATEN ISLAND,10308,NaN,NaN,N,NaN,...,40.550136,-74.150602,POINT (-74.1506024 40.5501364),503.0,51.0,013203,5065713,5051430020,SI0302,Great Kills-Eltingville
1,06/03/2024,2023,5005430010,271 BROAD STREET,STATEN ISLAND,10304,NaN,YES,N,NaN,...,40.623625,-74.083549,POINT (-74.0835487 40.6236254),501.0,49.0,002100,5166592,5005430010,SI0102,Tompkinsville-Stapleton-Clifton-Fox Hills
2,06/03/2024,2023,5051430017,3 NELSON AVENUE,STATEN ISLAND,10308,NaN,NaN,N,NaN,...,40.549870,-74.150609,POINT (-74.150609 40.5498701),503.0,51.0,013203,5065711,5051430017,SI0302,Great Kills-Eltingville
3,06/03/2024,2023,5006550014,1366 CLOVE ROAD,STATEN ISLAND,10301,NaN,YES,N,NaN,...,40.614009,-74.101917,POINT (-74.1019167 40.6140093),501.0,49.0,016901,5016536,5006550014,SI0105,Westerleigh-Castleton Corners
4,06/03/2024,2023,5001730034,693 HENDERSON AVENUE,STATEN ISLAND,10310,NaN,YES,N,NaN,...,40.637527,-74.115630,POINT (-74.1156299 40.6375275),501.0,49.0,010500,5108642,5001730034,SI0104,West New Brighton-Silver Lake-Grymes Hill


In [91]:
# Check column names
print(store_fronts.columns.tolist())

['Filing Due Date', 'Reporting Year', 'Borough Block Lot', 'Property Street Address or Storefront Address', 'Borough', 'Zip Code', 'Sold Date', 'Vacant on 12/31', 'Construction Reported', 'Vacant 6/30 or Date Sold', 'Primary Business Activity', 'Expiration date of the most recent lease', 'Property Number', 'Property Street', 'Unit', 'Borough1', 'Postcode', 'Latitude', 'Longitude', 'Lat/Long', 'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL', 'NTA', 'NTA Neighborhood']


In [ ]:
# haven't done it yet

# walkability

In [88]:
walk=pd.read_csv("data/walkability.csv")
walk.head()

,the_geom,OBJECTID,Loc,Borough,Street_Nam,From_Stree,To_Street,Iex,May07_AM,May07_PM,...,May23_MD,Oct23_AM,Oct23_PM,Oct23_MD,June24_AM,June24_PM,June24_MD,Oct24_AM,Oct24_PM,Oct24_MD
0,POINT (-73.90459140730678 40.87919896648574),1,1,Bronx,Broadway,West 231st Street,Naples Terrace,N,1189,4094,...,2496,935,3759,1745,901,3482,2052,1031,4097,2229.0
1,POINT (-73.92188432870218 40.82662794123292),2,2,Bronx,East 161st Street,Gra Concourse,Sheridan Avenue,Y,1511,3184,...,2064,2822,5050,1287,2113,4155,1493,3016,4685,1655.0
2,POINT (-73.89535781584335 40.86215460031517),3,3,Bronx,East Fordham Road,Valentine Avenue,Tiebout Avenue,Y,1832,12311,...,4696,1560,6880,2453,1485,7125,3819,1689,6405,4417.0
3,POINT (-73.87892467324478 40.8812869959873),4,4,Bronx,East Gun Hill Road,Bainbridge Avenue,Rochambeau Avenue,N,764,2673,...,1303,1404,2089,807,1394,2023,1190,1495,2474,1073.0
4,POINT (-73.88956389732787 40.844636776717664),5,5,Bronx,East Tremont Avenue,Prospect Avenue,Clinton Avenue,N,650,2538,...,1578,786,2140,1115,550,1455,1563,725,2329,1888.0


# BUS